In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain
from xsigmamodules.Random import random_enum
from xsigmamodules.Analytics import (
    calibrationIrTargetsConfiguration,
    correlationManager,
    calibrationHjmSettings,
    parameter_markovian_hjm_enum,
    calibrationIrHjm,
    parameterMarkovianHjmId,
    parameterMarkovianHjm,
    dynamicInstructionId,
    dynamicInstructionIrMarkovianHjm,
    simulatedMarketDataIrId,
    correlationManagerId,
    dynamicInstructionIrId,
    measureId,
    measure,
    randomConfig,
    simulationManager,
    randomConfigId,
)

from xsigmamodules.Util import dayCountConvention
from xsigmamodules.Vectorization import vector, matrix, tensor, tensor
from xsigmamodules.util.numpy_support import xsigmaToNumpy, numpyToXsigma
from xsigmamodules.common import helper
from xsigmamodules.market import market_data
from xsigmamodules.simulation import simulation
from xsigmamodules.util.misc import xsigmaGetDataRoot, xsigmaGetTempDir
from xsigmamodules.Market import (
    discountCurveInterpolated,
    discountCurveId,
    anyId,
    anyContainer,
    anyObject,
    irVolatilityDataSabr,
    discountCurveFlat,
)

# Folders

In [2]:
XSIGMA_DATA_ROOT = xsigmaGetDataRoot()
XSIGMA_TEST_ROOT = xsigmaGetTempDir()

In [3]:
print(XSIGMA_DATA_ROOT)

C:/dev/PRETORIAN/Testing


In [4]:
discount_id = discountCurveId("USD", "LIBOR.3M.USD")
diffusion_id = simulatedMarketDataIrId(discount_id)

discount_curve = discountCurveInterpolated.read_from_json(
    XSIGMA_DATA_ROOT + "/Data/discountCurve.json"
)
anyids = [anyId(discount_id)]
anyobject = [anyObject(discount_curve)]

correlation_mgr = correlationManager.read_from_json(
    XSIGMA_DATA_ROOT + "/Data/correlationManager.json"
)
anyids.append(anyId(correlationManagerId()))
anyobject.append(anyObject(correlation_mgr))

valuation_date = discount_curve.valuation_date()

target_config = calibrationIrTargetsConfiguration.read_from_json(
    XSIGMA_DATA_ROOT + "/Data/calibrationIrTargetsConfiguration.json"
)

ir_volatility_surface = irVolatilityDataSabr.read_from_json(
    XSIGMA_DATA_ROOT + "/Data/irVolatilityData.json"
)

In [5]:
diffusion_ids = [diffusion_id]
correlation = correlation_mgr.pair_correlation_matrix(diffusion_ids, diffusion_ids)

# Load data

In [6]:
# M_np = np.array([[1.0,	-0.5,  -0.5,	-0.5],
#                 [-0.5,	 1.0,	-0.5,	-0.5],
#                 [-0.5,	-0.5,	1.0,	-0.5],
#                 [-0.5,	-0.5,	-0.5,	1.0]], "d")
# correlation = numpyToXsigma(M_np)
# correlation_mgr = correlationManager(valuation_date, diffusion_ids, [4], correlation)

volatility_bounds = [0.0001, 1]
decay_bounds = [0.0001, 1.0]
calibration_settings = calibrationHjmSettings(
    correlation.rows(),
    volatility_bounds,
    decay_bounds,
    parameter_markovian_hjm_enum.PICEWISE_CONSTANT,
    True,
    200,
    False,
    False,
    1.0,
)
calibration_settings_aad = calibrationHjmSettings(
    correlation.rows(),
    volatility_bounds,
    decay_bounds,
    parameter_markovian_hjm_enum.PICEWISE_CONSTANT,
    True,
    200,
    True,
    False,
    1.0,
)
convention = dayCountConvention()

In [7]:
# print(target_config)

# Static data

In [8]:
mkt_data_obj = market_data.market_data(XSIGMA_DATA_ROOT)

In [9]:
calibrator = calibrationIrHjm(valuation_date, target_config)

# Run calibration

In [10]:
start = time.time()
parameter = calibrator.calibrate(
    parameterMarkovianHjmId(diffusion_id),
    calibration_settings_aad,
    discount_curve,
    ir_volatility_surface,
    correlation_mgr,
)
end = time.time()
m1 = end - start
print(end - start)

0.07255744934082031


In [11]:
start = time.time()
parameter = calibrator.calibrate(
    parameterMarkovianHjmId(diffusion_id),
    calibration_settings,
    discount_curve,
    ir_volatility_surface,
    correlation_mgr,
)
end = time.time()
m2 = end - start
print(end - start)

1.4948210716247559


In [12]:
m2 / m1

20.601896637838124

# Simulation

In [13]:
anyids.append(anyId(parameterMarkovianHjmId(diffusion_id)))
anyobject.append(anyObject(parameter))

anyids.append(anyId(dynamicInstructionIrId(diffusion_id)))
anyobject.append(anyObject(dynamicInstructionIrMarkovianHjm()))

anyids.append(anyId(measureId()))
anyobject.append(anyObject(measure(discount_id)))

num_of_paths = 262144 * 2
config = randomConfig(random_enum.SOBOL_BROWNIAN_BRIDGE, 12765793, num_of_paths)

anyids.append(anyId(randomConfigId()))
anyobject.append(anyObject(config))

market = anyContainer(anyids, anyobject)

simulation_dates = helper.simulation_dates(valuation_date, "3M", 120)

maturity = max(simulation_dates)

sim = simulation.Simulation(
    mkt_data_obj,
    num_of_paths,
    target_config.frequency(),
    target_config.expiries(),
    target_config.cms_tenors(),
    target_config.coterminal(),
    maturity,
    simulation_dates,
)

sim.run_simulation(diffusion_ids, market, simulation_dates)

KeyboardInterrupt: 

In [ ]:
sim.plot(simulation_dates)